In [3]:
#linux
#terminal
#conda install gdal

In [14]:
#import required libraries
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
from osgeo import gdal
import xml.etree.ElementTree as ET

In [117]:
def cut_plot(photo_dir_path, output_path, xmin, ymin, xmax, ymax):
    """
    photo_dir_path: sciezka do katalogu surowa_paczka.SAFE
    output_path: sciezka pliku wycietego zdjecia np.: zdjecie.png
    xmin, ymin, xmax, ymax: wspolrzedne prostokata do wyciecia
    """
    
    target_dir = os.listdir(photo_dir_path + "/GRANULE/")[0]
    
    #XML part
    root = ET.parse(photo_dir_path + "/GRANULE/" + target_dir + "/MTD_TL.xml").getroot()
    epsg_info = list(list(root)[1])[0].find("HORIZONTAL_CS_CODE").text[5:]
    
    
    photos_paths = os.listdir(photo_dir_path + "/GRANULE/" + target_dir + "/IMG_DATA/R10m")
    for f in photos_paths:
        if(f.find("TCI") != -1):
            photo_path = photo_dir_path + "/GRANULE/" + target_dir + "/IMG_DATA/R10m/" + f
            break
    
    #JP2 to TIFF
    gdal.Translate("temp_tiff.tiff", photo_path)
    
    #Wycinanie opcje
    opt = gdal.WarpOptions(srcSRS = 'epsg:'+epsg_info, dstSRS = 'epsg:4326', 
                       outputBounds = (xmin, ymin, xmax, ymax), format = "Gtiff")
    
    #Wycinanie
    res = gdal.Warp('usunac.tiff', 'temp_tiff.tiff', options = opt)
    
    #Wyciete do png
    gdal.Translate(output_path, res)
    
    #Usuwanie temp_tiff
    os.remove("temp_tiff.tiff")
    os.remove("usunac.tiff")
    os.remove(output_path + ".aux.xml")
    
    del res
    
    return

In [118]:
cut_plot("S2B_MSIL2A_20200527T095029_N0214_R079_T34UDC_20200527T123117.SAFE", 
        "nareszcie.png",
        20.18396806301617, 51.8500806996221,20.18664160370571,51.85258457896293)